In [1]:
import lightning as L
from slr.models.CorrNet import CorrNet
from slr.datasets.Phoenix2014DataModule import Phoenix2014DataModule
from slr.datasets.transforms import TemporalRescale, ToTensor
from torchvision.transforms import Compose, Resize, RandomCrop, RandomHorizontalFlip, CenterCrop, Normalize


In [2]:
# load model
model = CorrNet.load_from_checkpoint(
    "/new_home/xzj23/workspace/SLR/experiments/Phoenix2014_new/CorrNet/2024-10-09_23-11-41/checkpoints/epoch=38_DEV_WER=0.00.ckpt")

In [3]:
# get data
datamodule = Phoenix2014DataModule(
    dataset_dir="/new_home/xzj23/workspace/SLR/data/phoenix2014",
    features_dir="/new_home/xzj23/workspace/SLR/data/phoenix2014/phoenix-2014-multisigner/features/fullFrame-256x256px",
    batch_size=1, num_workers=10,
    transform={
        'train': Compose([ToTensor(), RandomCrop(224), RandomHorizontalFlip(0.5), TemporalRescale(0.2)]),
        'dev': Compose([ToTensor(), CenterCrop(224)]),
        'test': Compose([ToTensor(), CenterCrop(224)])
    },
    tokenizer=model.probs_decoder.tokenizer,
)
datamodule.setup(stage="test")

dl = datamodule.test_dataloader()

In [4]:
# trainer
trainer = L.Trainer(accelerator='gpu', devices=[0], precision="16-mixed", logger=False)
res = trainer.predict(model=model, dataloaders=dl)

/new_home/xzj23/.conda/envs/slr/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /new_home/xzj23/.conda/envs/slr/lib/python3.10/site- ...
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: 0it [00:00, ?it/s]

In [5]:
for item in res:
    print(f"{item[0][0]:<50}: {' '.join(item[1][0])}")

01April_2010_Thursday_heute_default-5             : ABER WOCHE MORGEN SONNE SELTEN REGEN
01April_2010_Thursday_tagesschau_default-7        : SAMSTAG WECHSELHAFT BESONDERS FREUNDLICH loc-NORDOST __EMOTION__ IX loc-ZONE
01April_2010_Thursday_tagesschau_default-8        : SONNTAG REGEN TEIL GEWITTER loc-SUEDOST DURCHGEHEND REGEN
01April_2011_Friday_tagesschau_default-0          : __ON__ JETZT WETTER WIE-AUSSEHEN MORGEN SAMSTAG ZWEITE APRIL __OFF__ ZEIGEN-BILDSCHIRM __OFF__
01April_2011_Friday_tagesschau_default-6          : __ON__ FLUSS NORDOSTRAUM HEUTE NACHT NOCH WOLKE KOENNEN SCHNEE REGEN REGION
01April_2011_Friday_tagesschau_default-14         : __ON__ MONTAG REGION WECHSELHAFT ABER KUEHL AB GERADE IN-KOMMEND ZURUECK FREUNDLICH WARM __OFF__
01December_2011_Thursday_heute_default-6          : SONST REGION MILD NEUN BIS VIERZEHN GRAD TAG AUCH REGEN
01December_2011_Thursday_tagesschau_default-6     : WIND MAESSIG FRISCH STURM STARK STURM BERG SCHWER STURM SUEDOSTRAUM SCHWACH WIND
01Febru